# Imports

In [1]:
import pandas as pd

C:\Users\ozzafar\AppData\Local\Temp\ipykernel_33248\2035458946.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Find best lambda

In [8]:
all_lambdas = pd.DataFrame(columns=['lambda', 'SD MAE (clipcount)', 'Ours MAE (clipcount)', 'SD MAE (YOLO)', 'Ours MAE (YOLO)', 'SD CLIP', 'Ours CLIP'])
lambdas = [0,5,25,100]

for _lambda in lambdas:
    curr = pd.read_pickle(f"experiments/experiment_{_lambda}lambda.pkl")
    curr = curr[curr['is_clipcount'] == True]
    all_lambdas = pd.concat([all_lambdas, pd.DataFrame
    ([
        {
            "lambda" : _lambda,
            'SD MAE (clipcount)' : curr['sd_count_diff'].mean(),
            'Ours MAE (clipcount)' : curr['sd_optimized_count_diff'].mean(),
            'SD MAE (YOLO)' : curr[curr['is_yolo'] == True]['sd_count_diff2'].mean(),
            'Ours MAE (YOLO)' : curr[curr['is_yolo'] == True]['sd_optimized_count_diff2'].mean(),
            'SD CLIP' : 1-curr['actual_relevance_score'].mean(),
            'Ours CLIP' : 1-curr['optimized_relevance_score'].mean()
        }
    ])], ignore_index=True)
    
all_lambdas

C:\Users\ozzafar\AppData\Local\Temp\ipykernel_33248\2454065537.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_lambdas = pd.concat([all_lambdas, pd.DataFrame


,lambda,SD MAE (clipcount),Ours MAE (clipcount),SD MAE (YOLO),Ours MAE (YOLO),SD CLIP,Ours CLIP
0,0,21.590128,6.435795,15.282051,12.051282,0.706846,0.706864
1,5,21.354273,5.118946,15.282051,14.615385,0.705741,0.705988
2,25,21.462005,5.052094,15.282051,11.435897,0.704020,0.702141
3,100,21.462005,5.640778,15.282051,12.666667,0.704020,0.702143


In [9]:
best_lambda_index = all_lambdas["Ours MAE (YOLO)"].idxmin()
best_lambda = lambdas[best_lambda_index]
best_lambda

25

In [10]:
best_lambda_df = pd.read_pickle(f"experiments/experiment_{best_lambda}lambda.pkl")
best_lambda_df = best_lambda_df[best_lambda_df['is_clipcount'] == True]

# SD (clipcount) MAE
sd_clipcount_mae = best_lambda_df.groupby('amount').agg({'sd_count_diff': 'mean'}).reset_index()
sd_clipcount_mae.rename(columns={'sd_count_diff': 'SD MAE (clipcount)'}, inplace=True)

# Ours (clipcount) MAE
ours_clipcount_mae = best_lambda_df.groupby('amount').agg({'sd_optimized_count_diff': 'mean'}).reset_index()
ours_clipcount_mae.rename(columns={'sd_optimized_count_diff': 'Ours MAE (clipcount)'}, inplace=True)

# SD (yolo) MAE
sd_yolo_mae = best_lambda_df[best_lambda_df['is_yolo'] == True].groupby('amount').agg({'sd_count_diff2': 'mean'}).reset_index()
sd_yolo_mae.rename(columns={'sd_count_diff2': 'SD MAE (YOLO)'}, inplace=True)

# Ours (yolo) MAE
ours_yolo_mae = best_lambda_df[best_lambda_df['is_yolo'] == True].groupby('amount').agg({'sd_optimized_count_diff2': 'mean'}).reset_index()
ours_yolo_mae.rename(columns={'sd_optimized_count_diff2': 'Ours MAE (YOLO)'}, inplace=True)

# SD (CLIP)
sd_clip = (1 - best_lambda_df.groupby('amount').agg({'actual_relevance_score': 'mean'})).reset_index()
sd_clip.rename(columns={'actual_relevance_score': 'SD CLIP'}, inplace=True)

# Ours (CLIP)
ours_clip = (1 - best_lambda_df.groupby('amount').agg({'optimized_relevance_score': 'mean'})).reset_index()
ours_clip.rename(columns={'optimized_relevance_score': 'Ours CLIP'}, inplace=True)

# Merge all dataframes
merged_df = sd_clipcount_mae.merge(ours_clipcount_mae, on='amount', how='outer')
merged_df = merged_df.merge(sd_yolo_mae, on='amount', how='outer')
merged_df = merged_df.merge(ours_yolo_mae, on='amount', how='outer')
merged_df = merged_df.merge(sd_clip, on='amount', how='outer')
merged_df = merged_df.merge(ours_clip, on='amount', how='outer')

merged_df

,amount,SD MAE (clipcount),Ours MAE (clipcount),SD MAE (YOLO),Ours MAE (YOLO),SD CLIP,Ours CLIP
0,5,10.79519,3.705284,4.384615,4.692308,0.701278,0.699034
1,15,22.651066,4.678397,18.538462,13.461538,0.706248,0.703396
2,25,30.939759,6.7726,22.923077,16.153846,0.704534,0.703994


# Explore best lambda

### 1. Compare with ControlNet Baseline

In [5]:
controlnet = pd.read_pickle(f"experiments/experiment_controlnet_1.pkl")
controlnet = controlnet[controlnet['is_clipcount'] == True]

sd_mae_clipcount = best_lambda_df['sd_count_diff'].mean()
controlnet_mae_clipcount = controlnet['sd_optimized_count_diff'].mean()
ours_mae_clipcount = best_lambda_df['sd_optimized_count_diff'].mean()

sd_mae_yolo = best_lambda_df[best_lambda_df['is_yolo']==True]['sd_count_diff2'].mean()
controlnet_mae_yolo = controlnet[controlnet['is_yolo']==True]['sd_optimized_count_diff2'].mean()
ours_mae_yolo = best_lambda_df[best_lambda_df['is_yolo']==True]['sd_optimized_count_diff2'].mean()

sd_clip = 1-best_lambda_df['actual_relevance_score'].mean()
controlnet_clip = 1-controlnet['optimized_relevance_score'].mean()
ours_mae_clip = 1-best_lambda_df['optimized_relevance_score'].mean()


data = {
    'MAE (clipcount)': [sd_mae_clipcount, controlnet_mae_clipcount, ours_mae_clipcount],
    'MAE (YOLO)': [sd_mae_yolo, controlnet_mae_yolo, ours_mae_yolo],
    'CLIP': [sd_clip, controlnet_clip, ours_mae_clip]
}

pd.DataFrame(data, index=['SD', 'ControlNet', 'Ours'])

,MAE (clipcount),MAE (YOLO),CLIP
SD,21.462005,15.282051,0.704020
ControlNet,9.812914,15.000000,0.720257
Ours,5.052094,11.435897,0.702141


## Compare to standard SDXL with different number of diffusion steps

In [11]:
all_steps = pd.DataFrame(columns=['steps', 'SD (clipcount) MAE', 'Ours (clipcount) MAE', 'SD (yolo) MAE', 'Ours (yolo) MAE', 'SD (CLIP)', 'Ours (CLIP)'])

for step in range(1,6):
    curr = pd.read_pickle(f"experiments/experiment_{best_lambda}lambda-eval-{step}.pkl")
    curr = curr[curr['is_clipcount'] == True]
    all_steps = pd.concat([all_steps, pd.DataFrame
    ([
        {
            "steps" : step,
            'SD (clipcount) MAE' : curr['sd_count_diff'].mean(),
            'Ours (clipcount) MAE' : curr['sd_optimized_count_diff'].mean(),
            'SD (yolo) MAE' : curr[curr['is_yolo'] == True]['sd_count_diff2'].mean(),
            'Ours (yolo) MAE' : curr[curr['is_yolo'] == True]['sd_optimized_count_diff2'].mean(),
            'SD (CLIP)' : 1-curr['actual_relevance_score'].mean(),
            'Ours (CLIP)' : 1-curr['optimized_relevance_score'].mean()
        }
    ])], ignore_index=True)
    
all_steps

C:\Users\ozzafar\AppData\Local\Temp\ipykernel_33248\1922994017.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_steps = pd.concat([all_steps, pd.DataFrame


,steps,SD (clipcount) MAE,Ours (clipcount) MAE,SD (yolo) MAE,Ours (yolo) MAE,SD (CLIP),Ours (CLIP)
0,1,21.462005,21.462005,15.282051,15.282051,0.70402,0.704020
1,2,21.462005,15.623152,15.282051,14.871795,0.70402,0.704414
2,3,21.462005,14.957330,15.282051,15.692308,0.70402,0.707320
3,4,21.462005,14.929599,15.282051,16.743590,0.70402,0.708308
4,5,21.462005,14.656707,15.282051,15.307692,0.70402,0.709795
